In [1]:
!pip install html5lib
!pip install tqdm
!pip install pandas
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import html5lib
import time


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Rismosch\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
url = "https://us.soccerway.com/international/world/world-cup/2022-qatar/group-stage/r49519/"
group_div = "block_competition_tables_group-wrapper"

my_agent = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15"}
response = requests.get(url, headers = my_agent)
soup = BeautifulSoup(response.text)

all_current = soup.find_all("div", {"class": group_div})

team_list = []

for div in all_current:
    
    tables = div.findChildren("table", recursive=True)
    table = tables[0]
    
    trs = table.findChildren("tr", recursive=True)

    for tr in trs:
        tds = tr.findChildren("td", recursive=True)
        for td in tds:
            classes = td.get("class")
            if 'large-link' in classes:
                team_name = td.findChildren("a")[0].contents[0]
                href = td.findChildren("a")[0]["href"]
                
                base_url = "https://us.soccerway.com"
                link = f'{base_url}{href}'
                
                team = {
                    "name": team_name,
                    "link": link,
                }
                
                team_list.append(team)
            
for team in team_list:
    print(f'{team["link"]} {len(team["link"])}')

# pandas truncates string longer than 50 characters by default
# but some of our links are longer than 50 characters
# we set the length to 2048, the max URL length
pd.options.display.max_colwidth = 2048
            
data_frame = pd.DataFrame(data=team_list)
print(data_frame)

https://us.soccerway.com/teams/netherlands/netherlands/1552/ 60
https://us.soccerway.com/teams/ecuador/ecuador/657/ 51
https://us.soccerway.com/teams/senegal/senegal/1941/ 52
https://us.soccerway.com/teams/qatar/qatar/1774/ 48
https://us.soccerway.com/teams/england/england/774/ 51
https://us.soccerway.com/teams/iran/iran/1178/ 46
https://us.soccerway.com/teams/united-states/united-states-of-america/2281/ 75
https://us.soccerway.com/teams/wales/wales/2332/ 48
https://us.soccerway.com/teams/saudi-arabia/saudi-arabia/1897/ 62
https://us.soccerway.com/teams/poland/poland/1677/ 50
https://us.soccerway.com/teams/mexico/mexico/1497/ 50
https://us.soccerway.com/teams/argentina/argentina/132/ 55
https://us.soccerway.com/teams/france/france/944/ 49
https://us.soccerway.com/teams/tunisia/tunisia/2211/ 52
https://us.soccerway.com/teams/denmark/denmark/643/ 51
https://us.soccerway.com/teams/australia/australia/156/ 55
https://us.soccerway.com/teams/spain/spain/2137/ 48
https://us.soccerway.com/team

In [3]:
def get_points_from_tr(tr):
    tds = tr.findChildren('td');
    result = tds[3]

    _as = result.findChildren('a')
    a = _as[0]

    result_classes = a.get("class")

    return calculate_points(result_classes)
    
    

def calculate_points(classes):
    # cancelled games wont have any classes.
    # we skip the point calculation in that
    # case, because the next line would throw
    # an "index out of range" error if we would
    # run it.
    print(classes)
    if not classes:
        return 0
        
    result_class = classes[0]

    if result_class == "result-win":
        return 3
    elif result_class == "result-draw":
        return 1
    
    return 0

point_list = []
woc_point_list = []

# calculate points for every team
for team in team_list:
    match_url = f'{team["link"]}matches/'

    print(match_url)

    # sleep to not overwhelm server
    time.sleep(1)
    
    response = requests.get(match_url, headers = my_agent)
    soup = BeautifulSoup(response.text)

    tables = soup.findAll("table", recursive=True)
    table = tables[0]

    tbody = table.findChildren('tbody')[0]
    trs = tbody.findChildren('tr');

    # find index of first WOC match
    index = 0;
    for tr in trs:
        tds = tr.findChildren('td')
        match_type_cell = tds[1]
        _as = match_type_cell.findChildren('a')
        _a = _as[0]
        match_type = _a.contents[0]

        if match_type == "WOC":
            break

        index += 1

    # get points of first WOC match
    woc_points = get_points_from_tr(trs[index])
    woc_point_list.append(woc_points)
    
    
    # find points of previous 5 matches
    amount = 5;
    index -= 1;
    indecies = range(index, index - amount, -1)

    points = 0
    for i in indecies:
        points += get_points_from_tr(trs[i])

    point_list.append(points);
        
data_frame['points'] = point_list
data_frame['woc_points'] = woc_point_list

print(data_frame)

https://us.soccerway.com/teams/netherlands/netherlands/1552/matches/
['result-win']
['result-win']
['result-win']
['result-win']
['result-draw']
['result-win']
https://us.soccerway.com/teams/ecuador/ecuador/657/matches/
['result-win']
['result-draw']
['result-draw']
['result-draw']
['result-win']
['result-draw']
https://us.soccerway.com/teams/senegal/senegal/1941/matches/
['result-loss']
['result-draw']
['result-win']
['result-win']
['result-win']
['result-loss']
https://us.soccerway.com/teams/qatar/qatar/1774/matches/
['result-loss']
['result-win']
['result-win']
['result-win']
['result-win']
['result-draw']
https://us.soccerway.com/teams/england/england/774/matches/
['result-win']
['result-draw']
['result-loss']
['result-loss']
['result-draw']
['result-draw']
https://us.soccerway.com/teams/iran/iran/1178/matches/
['result-win']
['result-win']
['result-loss']
[]
['result-win']
['result-loss']
https://us.soccerway.com/teams/united-states/united-states-of-america/2281/matches/
['result-

In [40]:
url = "https://us.soccerway.com/teams/rankings/fifa/"

response = requests.get(url, headers = my_agent)
soup = BeautifulSoup(response.text)

all_current = soup.find_all("table")
table = all_current[0]
tbody = table.findChildren("tbody", recursive=True)[0]
trs = tbody.findChildren("tr")

rank_data_list = []
for tr in trs:
    tds = tr.findChildren("td");

    rank_td = tds[0]
    team_td = tds[1]
    points_td = tds[2]
    
    rank = int(rank_td.contents[0])
    
    team_a = team_td.findChildren("a")[0]
    team = team_a.contents[0]
    
    points = float(points_td.contents[0])
    
    rank_data = {
        "rank": rank,
        "team": team,
        "points": points
    }
    
    rank_data_list.append(rank_data)

# counter = 0
# for rank_data in rank_data_list:
#     print(f'{counter} {rank_data}')
#     counter += 1

rank_list = []
points_list = []

for team in team_list:
    
    index = 0;
    for rank_data in rank_data_list:
        if team["name"] == rank_data["team"]:
            break
        
        index += 1
    
    found_rank_data = rank_data_list[index] 
    rank = found_rank_data["rank"]
    points = found_rank_data["points"]
    
    rank_list.append(rank)
    points_list.append(points)
    
data_frame['rank'] = rank_list
data_frame['score'] = points_list

print(data_frame)

0 {'rank': 1, 'team': 'Brazil', 'points': 1841.3}
1 {'rank': 2, 'team': 'Belgium', 'points': 1816.71}
2 {'rank': 3, 'team': 'Argentina', 'points': 1773.88}
3 {'rank': 4, 'team': 'France', 'points': 1759.78}
4 {'rank': 5, 'team': 'England', 'points': 1728.47}
5 {'rank': 6, 'team': 'Italy', 'points': 1726.14}
6 {'rank': 7, 'team': 'Spain', 'points': 1715.22}
7 {'rank': 8, 'team': 'Netherlands', 'points': 1694.51}
8 {'rank': 9, 'team': 'Portugal', 'points': 1676.56}
9 {'rank': 10, 'team': 'Denmark', 'points': 1666.57}
10 {'rank': 11, 'team': 'Germany', 'points': 1650.21}
11 {'rank': 12, 'team': 'Croatia', 'points': 1645.64}
12 {'rank': 13, 'team': 'Mexico', 'points': 1644.89}
13 {'rank': 14, 'team': 'Uruguay', 'points': 1638.71}
14 {'rank': 15, 'team': 'Switzerland', 'points': 1635.92}
15 {'rank': 16, 'team': 'United States', 'points': 1627.48}
16 {'rank': 17, 'team': 'Colombia', 'points': 1611.04}
17 {'rank': 18, 'team': 'Senegal', 'points': 1584.38}
18 {'rank': 19, 'team': 'Wales', 'poi